# Generating Audio with LSTM 

This notebook will let you download a song from Youtube and model it with an **LSTM**

This models the way each spectral frame follows another and can be used to generate new raw audio

## Install prerequisites and get code

In [1]:
!pip install -U numpy==1.19.5

     |████████████████████████████████| 14.8 MB 14.2 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
tensorflow 2.8.0 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


^^Restart runtime if required!^^

In [1]:
%tensorflow_version 1.x
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # less warnings ...
import tensorflow as tf

TensorFlow 1.x selected.


In [2]:
!git clone https://github.com/ual-cci/music_gen_interaction_RTML.git

Cloning into 'music_gen_interaction_RTML'...
remote: Enumerating objects: 590, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 590 (delta 0), reused 0 (delta 0), pack-reused 589
Receiving objects: 100% (590/590), 15.75 MiB | 7.16 MiB/s, done.
Resolving deltas: 100% (370/370), done.


In [3]:
# python libraries
!pip install Pillow opencv-python PyWavelets tqdm slugify
!pip install -U Flask
!pip install lws==1.2.6
!pip install tflearn
!pip install librosa==0.7.2
!pip install numba==0.48
!pip install mock

  Created wheel for slugify: filename=slugify-0.0.1-py3-none-any.whl size=1908 sha256=72bc92f5d3e3eec378eac2224077a0b51e80b3cd03ed267fe9eaad20b5dba552
  Stored in directory: /root/.cache/pip/wheels/d4/7b/0d/bd65011a1b442843bb4043e396f727ab0f1e76050355b9156a
Successfully built slugify
     |████████████████████████████████| 95 kB 4.4 MB/s 
     |████████████████████████████████| 289 kB 68.8 MB/s 
     |████████████████████████████████| 133 kB 75.1 MB/s 
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 1.0.1
    Uninstalling Werkzeug-1.0.1:
      Successfully uninstalled Werkzeug-1.0.1
  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
  Attempting uninstall: itsdangerous
    Found existing installation: itsdangerous 1.1.0
    Uninstalling itsdangerous-1.1.0:
      Successfully uninstalled itsdangerous-1.1.0
  Attempting uninstall: Flask
    Found existing ins

In [4]:
%cd /content/music_gen_interaction_RTML

/content/music_gen_interaction_RTML


## Download a sample audio:

Note: replace the url with whatever music video you want - or upload a file directly ... You can use the ffmpeg to convert it to wav later.

In [5]:
# get a youtube downloader
!sudo curl -L https://yt-dl.org/downloads/latest/youtube-dl -o /usr/local/bin/youtube-dl
!sudo chmod a+rx /usr/local/bin/youtube-dl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100     3  100     3    0     0     24      0 --:--:-- --:--:-- --:--:--    24
100     3  100     3    0     0     22      0 --:--:-- --:--:-- --:--:--    22
100   645  100   645    0     0   3816      0 --:--:-- --:--:-- --:--:--  3816
100 1794k  100 1794k    0     0  3245k      0 --:--:-- --:--:-- --:--:-- 3245k


# Set up training

In [6]:
%cd /content/music_gen_interaction_RTML/

from unittest.mock import Mock, MagicMock
args = MagicMock(name='method')
sample_rate = 44100

# This will set the same setting for the training and the running of the model:
args.lstm_layers = 3
args.lstm_units = 128
args.sample_rate = sample_rate
args.sequence_length = 40
args.async_loading = True
args.amount_epochs = -1

/content/music_gen_interaction_RTML


# Pick a new song from Youtube 

## Insert your own url below and train 



In [7]:
youtube_url = "https://www.youtube.com/watch?v=1mTsu8HvC1g"

!youtube-dl -ci -f "bestaudio[ext=m4a]" $youtube_url -o 'youtube_audio.m4a'
!ffmpeg -i 'youtube_audio.m4a' -ac 2 -f wav full.wav

[youtube] 1mTsu8HvC1g: Downloading webpage
[download] Destination: youtube_audio.m4a
[download] 100% of 2.02MiB in 00:30
[ffmpeg] Correcting container in "youtube_audio.m4a"
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh 

In [8]:
# Audio file preparation - this cuts a 1 minute sample from the audio file:
!ffmpeg -y -ss 60 -i full.wav -t 60 -c copy sample.wav

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

### You will probably need to train for 300 epochs

In [9]:
number_of_epochs = 150 # will take cca 4min
number_of_epochs = 300 # will take cca 8min

song_name = "sample.wav" # < we will train the model on this wav file
model_name = "my_trained_model" # < and then save the model under this name (check the .tfl files in the folder afterwards)

# Train Model

In [10]:
# ----[keep the same bellow]-------------------------------------------------------------------

# takes time!
!python training_handler.py -target_file $song_name -model_name $model_name -amount_epochs $number_of_epochs -batch_size 512 \
                            -lstm_layers $args.lstm_layers  -lstm_units $args.lstm_units -sample_rate $args.sample_rate -sequence_length $args.sequence_length
from IPython.display import clear_output 
# clear_output()

# import glob
# import numpy as np
# i = glob.glob("*.tfl.png")
# from IPython.display import Audio, Image
# #display(Image(i[0]))

Streaming output truncated to the last 5000 lines.
Training Step: 916  | total loss: 0.04691 | time: 0.820s
| Adam | epoch: 084 | loss: 0.04691 - acc: 0.4672 -- iter: 1536/5208
Training Step: 917  | total loss: 0.04718 | time: 1.100s
| Adam | epoch: 084 | loss: 0.04718 - acc: 0.4666 -- iter: 2048/5208
Training Step: 918  | total loss: 0.04727 | time: 1.388s
| Adam | epoch: 084 | loss: 0.04727 - acc: 0.4711 -- iter: 2560/5208
Training Step: 919  | total loss: 0.04688 | time: 1.669s
| Adam | epoch: 084 | loss: 0.04688 - acc: 0.4732 -- iter: 3072/5208
Training Step: 920  | total loss: 0.04672 | time: 1.942s
| Adam | epoch: 084 | loss: 0.04672 - acc: 0.4741 -- iter: 3584/5208
Training Step: 921  | total loss: 0.04649 | time: 2.232s
| Adam | epoch: 084 | loss: 0.04649 - acc: 0.4761 -- iter: 4096/5208
Training Step: 922  | total loss: 0.04634 | time: 2.506s
| Adam | epoch: 084 | loss: 0.04634 - acc: 0.4801 -- iter: 4608/5208
Training Step: 923  | total loss: 0.04566 | time: 2.716s
| Adam | e

# Generate

As we mentioned in the lecture, we start from a place in the original audio track, then use the model to keep predicting new audio frames. 

Unfortunately, sometimes it can get stuck in a loop, or just stop generating interesting things so we can give it a kick by jumping to a new point in the song. 

Once we've switched, we keep using the model to generate new audio frames!

Below, you can specify a sequence of lengths and changes to the generation process to create a new audio output. 

The `sequence` array stores these points. Each item in is an array that stores the **start position** and **segment length**. The default has three, but you can add in as many as you want and/or change the existing values. 

```
sequence = [
  [start1, length1],
  [start2, length2],
  [start3, length3],
  etc....
]
```


The default code:

* Starts generating 10% through the song and generates 200 frames 

* Moves to 60% through the song and generates 300 frames

* Moves to 90% through the song and generates 150 frames

**Try with your own!**

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
# Here you can change the paths to other trained models / other song files
load_generative_seeds_from = song_name
load_model_from = model_name+".tfl"

In [13]:
from server_handler import ServerHandler
import settings

my_settings = settings.Settings(args)
print("Important Settings: settings.lstm_layers=", my_settings.lstm_layers, ", settings.lstm_units=", my_settings.lstm_units,
              ", settings.sample_rate=", my_settings.sample_rate)

generation_handler = ServerHandler(my_settings, manual_loading = True)
generation_handler.manual_init_song_model(load_generative_seeds_from, load_model_from)

# slightly experimental interpolation through the latents while generating ...

generation_handler.change_impulse(0.2) # set to 20% sharp

sequence = [
  #Starts generating 10% through the song and generates 200 frames 
  [0.1, 200],
  #Moves to 60% through the song and generates 300 frames
  [0.6, 300],
  #Moves to 90% through the song and generates 150 frames
  [0.9, 150],          
]

output_audio = []

for i in sequence:
  position_in_the_song = i[0]
  requested_length = i[1]
  generation_handler.change_impulse_smoothly_start(position_in_the_song) # allow interpolation
  audio, predict, reconstruct = generation_handler.generate_audio_sample(requested_length, interactive_i=position_in_the_song)
  output_audio.append(audio)

clear_output()

# Play Audio

If you want to download the audio, you can use the Colab File Explorer on the left <----. 

Find the file `generated_output_exp_concat.wav` and select **download**

In [14]:
import librosa
#\dc"Kca'lms'M
import numpy as np

In [15]:
from IPython.display import Audio, Image
output_audio = np.concatenate(output_audio)

out_name = 'generated_output_exp_concat.wav'
librosa.output.write_wav(out_name, output_audio, sr=sample_rate)
Audio(out_name)

# Clean up if you are going to train a new model from a different youtube video

In [ ]:
# fast cleanup
!mkdir unused
!mv *.wav unused/
!mv *.m4a unused/